In [4]:
# the purpose here is to get the region of interest of the extracted e's
# and calculate the GLCM of the pixels slices received, 

# acc. to paper the pixels recieved are 180x160, but we receive 14x14, lol so that's one thing
# output preferebly will be a feature vector having the GLCM features for this one pic

In [100]:
# A. Marginal Probabalities - GLCM - DONE
#     a. mean of row
#     b. mean of column
#     c. variance of row
#     d. variance of column
# B. Direct GLCM Metrics - GLCM
#     a. Energy (sum of squares of marginal probs) E - DONE
#     b. hxy1: Cross entropy using marginal product CEn
#     c. hxy2: Entropy of marginal product MEn
#     d. hglcm: Entropy of the GLCM itself EN
#     e. Max prob in GLCM values P_max
#     f. correlation C
#     g. diagonal correlation C_d
# C. From Difference Matrices - Diff GLCM
#     a. Energy E_d
#     b. Entropy En_d
#     c. Inertia I_d
#     d. Local Homogenteity H_d
# D. From Sum Matrices - Sum GLCM
#     a. Energy E_s
#     b. Entropy En_s
#     d. Variance V_s
#     d. cluster shade
#     e. cluster prominence

In [119]:
import cv2
import pytesseract
import numpy as np
from PIL import Image, ImageDraw
import os

# GLCM
from skimage.feature import graycomatrix, graycoprops

In [120]:
# def get_e(image_path, n, crop_size, op_dir):
    
#     # Load the image
#     # image_path = 'test_images/test_0.png'  # Replace with your image
#     img1 = cv2.imread(image_path)
#     if img1 is None:
#         raise ValueError(f"Image at path {image_path} could not be loaded.")
            
#     img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
#     #upscalling images, solely to help tesseract, image extraction will be done from the og image
#     # img = cv2.resize(img2, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
#     img = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_CUBIC)   # this is the processed image which 
#                                                                                            # we put in the tesseract
#                                                                                            # coz it needs clearer pcitures to 
#                                                                                            # accurately locate the e's
#     # print(img.shape, img2.shape)
#     half = crop_size//2
    
#     # Get image height (Tesseract origin is bottom-left)
#     h, w = img.shape
    
#     # op_dir = 'saved_e'
#     os.makedirs(op_dir, exist_ok=True)
    
#     # Draw setup
#     # draw = ImageDraw.Draw(img)
    
#     # Get character bounding boxes
#     boxes = pytesseract.image_to_boxes(img)
#     count = 0

#     # Draw green dots on top of each 'e'
#     for b in boxes.strip().splitlines():
#         b = b.split()
#         char, x1, y1, x2, y2 = b[0], int(b[1]), int(b[2]), int(b[3]), int(b[4])
        
#         # Flip y-coordinates
#         y1_new = h - y1
#         y2_new = h - y2
    
#         # make a bounding box around the e
#         # cv2.rectangle(img, (x1, y2_new), (x2, y1_new), (255, 0, 0), 1)
    
#         if char.lower() == 'e' and count<n:
    
#             # compute center of bounding box    
#             cx = (x1 + x2) // 2
#             cy = (y1_new + y2_new) // 2
        
#             # Get top-left and bottom-right coordinates of fixed crop
#             x_start = max(0, cx - half)
#             y_start = max(0, cy - half)
#             x_end = min(img2.shape[1], cx + half)
#             y_end = min(img2.shape[0], cy + half)
    
#             # crop image
#             cropped = img2[y_start:y_end, x_start:x_end]
    
#             # if dimensions okay, at image to output dir, 
#             # later make sure that the image dirs are constant, else will cause computation error
#             if cropped.shape[0] > 0 and cropped.shape[1] > 0:
#                 out_path = os.path.join(op_dir, f"e_{count+1}.png")
#                 cv2.imwrite(out_path, cropped)
#                 count += 1

#     cv2.imwrite("output_with_boxes.png", img)
#     print(f"Saved {count} 'e' characters in '{op_dir}/' and annotated image as 'output_with_boxes.png'")
    
#     # # to check the dimensions of the images
#     # for filename in os.listdir(op_dir):
#     #     full_path = os.path.join(op_dir, filename)
#     #     img_saved = cv2.imread(full_path)
        
#     #     if img_saved is not None:
#     #         print(f"{filename}: {img_saved.shape}")
#     #     else:
#     #         print(f"Failed to read {filename}")

#     return 0

In [162]:
# get region of interest

# threshold -> if sufficiently black, it is counted as ROI
def roi(img, threshold):
    if len(img.shape) != 2:
        raise ValueError("Input image must be grayscale")
    R = np.sum(img < threshold)
    
    print(R)
    return R

In [163]:
# lets get the dimensions correct here
# GLCM's are inherenetly 4Ds - distance, angles, nxm pairs and number of pair occurances
# we are eliminating angle, hence we get a 3D vector -> (G,G,F), G=levels, D=distances
# 3D vector (GLCM GxGxD) -> normalise -> 2 x row/column sums (1 x 10)-> 2 x mean and variance of each row/column sum(1 x scalar)
# total = 4 features

In [188]:
def glcm(img_path, dist, levels):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Quantize to 8 gray levels (0 to 7)
    levels = 8
    quantized = (gray / (256 // levels)).astype(np.uint8)

    distance = list(range(1, dist+1))
    R = roi(gray, 256)
    
    # Compute GLCM
    glcm = graycomatrix(quantized, distances=distance, angles=[0], levels=levels, symmetric=False, normed=False)
    
    # display GLCM matrix for angle=0 and distance=1
    # print(glcm[:, :, 0, 0])
    n_glcm = glcm[:,:,:,0]  #//R 
    # print(n_glcm)
    return n_glcm

In [189]:
def get_sum(arr,ch):
    # print(arr.shape)
    if ch == 'c':
        c = np.sum(arr, axis=0, keepdims=True)
        # print("Shape is:", c.shape)
        return c 
    elif ch == 'r':
        r = np.sum(arr, axis=1, keepdims=True) 
        # print("Shape is:", r.shape)
        return r

In [195]:
def get_var(arr, mean):
    m, n, d = arr.shape

    if m!=1 and n ==1: 
        a = m
        axis = 0
    elif n!=1 and m ==1: 
        a = n
        axis = 1
    else: print("not valid state")

    var = np.zeros((1,1,d))
    mean = mean.flatten()
    
    for i in range(d):
        summ = 0
        for j in range(a):
            val = arr[j, 0, i] if axis == 0 else arr[0, j, i]
            summ += (j ** 2) * val
        var[0, 0, i] = summ - mean[i]
    return var

In [196]:
if __name__ == "__main__":
    img_path = "/home/raginivi/Desktop/CV_project/saved_e/e_1.png"  

    N = 256
    # get probablities from GLCM, this is nromalised with ROI
    p_row = get_sum(glcm(img_path, 10, N), 'r')
    p_col = get_sum(glcm(img_path, 10, N), 'c')
    print("Row Sums:", p_row) 
    print("Columns Sums:", p_col) 

    # added all the probablities such that the matrix is 1 scalar x no. of disances, (1 x 1 x 10)
    u_row = get_sum(p_row,'c')
    u_col = get_sum(p_col,'r')
    # print("r", u_row.shape)
    # print("c", u_col.shape)

    print("Mean of Row Sums:", u_row) 

    print("Mean of Column Sums:", u_col )
    print("Variance of Row Sums:", get_var(p_row, u_row)) # <- v_r
    print("Variance of Column Sums:", get_var(p_col, u_col)) # <- v_c


196
196
Row Sums: [[[ 18  18  17  15  14  14  14  14  14  12]]

 [[  5   5   4   2   2   2   2   2   1   1]]

 [[  8   8   6   5   4   4   4   3   3   1]]

 [[ 11  11  11  10   9   7   5   3   1   0]]

 [[  5   5   5   5   5   4   3   3   1   1]]

 [[  5   5   4   4   2   2   2   2   2   0]]

 [[  9   9   7   6   6   6   5   5   5   4]]

 [[121 107 100  93  84  73  63  52  43  37]]]
Columns Sums: [[[ 20  20  16   8   6   6   6   6   6   5]
  [  7   7   6   6   6   5   5   5   5   5]
  [  9   9   8   8   6   6   5   5   5   4]
  [ 11  11  11  11  10   8   6   4   2   1]
  [  6   6   6   5   5   3   3   2   1   1]
  [  5   5   5   5   3   3   3   3   3   1]
  [ 10   8   7   6   5   5   5   4   4   4]
  [114 102  95  91  85  76  65  55  44  35]]]
Mean of Row Sums: [[[182 168 154 140 126 112  98  84  70  56]]]
Mean of Column Sums: [[[182 168 154 140 126 112  98  84  70  56]]]
Variance of Row Sums: [[[6412. 5740. 5305. 4925. 4435. 3876. 3330. 2783. 2305. 1922.]]]
Variance of Column Sums: [[

In [197]:
def get_energy(arr):
   return np.sum(matrix ** 2)